<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging/human_present/human_present_generate_tags_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO v3 pre-trained on Google Open Images to add human present tags to EOL images
---
*Last Updated 5 January 2025*  
-Runs in Python 3 with Darknet and YOLOv3-   

Using a YOLOv3 model (downloaded from [here](https://github.com/AlexeyAB/darknet) ) pre-trained on [Google Open Images](https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F03vt0) as a method to do customized, large-scale image processing. EOL Chiroptera (bat) images will be tagged for human(s) present (body, eye, head, hand, foot, face, arm, leg ear, eye, face, nose, beard) using object detection. Tags will further extend EOLv3 image search functions.

Notes:   
* Run code blocks by pressing play button in brackets on left
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

References:   
* Check out [AlexeyAB's darknet repo](https://github.com/AlexeyAB/darknet) for Colab tutorials like [this one](https://colab.research.google.com/drive/12QusaaRj_lUwCGDvQNfICpa7kA7_a2dE).

## Installs & Imports
---

In [ ]:
#@title Choose where to save results
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your project wd in form field on right
basewd = "/content/drive/MyDrive/train" #@param ["/content/drive/MyDrive/train"] {allow-input: true}

# Type in the folder that you want to contain TF2 files
folder = "darknet" #@param ["darknet"] {allow-input: true}
cwd = basewd + '/' + folder

# Download helper_funcs folder
!pip3 -q install --upgrade gdown
!gdown 1-BaybePbv810CTuHrF8JGRJfuEsHRQYH
!tar -xzvf helper_funcs.tar.gz -C .

# Install requirements.txt
!pip3 -q install -r requirements.txt
print('\n\n\n \033[91m If ERROR from pip dependency solver listed above, please check through conflicting version dependencies and/or open an issue on the CV for EOL images Github: https://github.com/aubricot/computer_vision_with_eol_images/issues. \033[0m')

# Set up directory structure & make darknet
!python setup.py $cwd $basewd

In [ ]:
#@title Import libraries

# For importing/exporting files, working with arrays, etc
import glob
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import numpy as np
import csv
import time
import pandas as pd

# For downloading images
!apt-get -qq install aria2

# For drawing onto and plotting images
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sys.path.append('/content')

# Define EOL CV custom functions
from wrangle_data import *

## Generate tags for images
---
Run EOL 20k image bundles through pre-trained object detection models and save results in 4 batches (A-D) of 5000 images each.

In [ ]:
#@title Test: Run with sample EOL Chiroptera image (To test with your own image, upload file to data/imgs and update fn formfield)

# Run with sample EOL image
# Define image filepath and name
fn = "18.032f7703433402aa15bdccae63f5e94c.260x190.jpg" #@param ["18.032f7703433402aa15bdccae63f5e94c.260x190.jpg"] {allow-input: true}
img_fpath = 'data/imgs/' + fn

# Download image
%cd $cwd
%cd data/imgs
!gdown 10Ov02YgnjJo0gSQ0MC7B2o0WiJPl8rvN
%cd $cwd

# Run darknet and show bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights {img_fpath}

# Display detection results
print("\nObject detection results:")
imShow('predictions.jpg')

### Generate tags: Run inference on EOL images & save results for tagging - Run 4X for batches A-D
Use 20K EOL Chiroptera image bundle to identify humans present. Results are saved to [tags_file].tsv. Run this section 4 times (to make batches A-D) of 5K images each to incrementally save in case of Colab timeouts.

In [ ]:
#@title Enter EOL image bundle and choose inference settings. Change **tags_file** for each batch A-D
%cd $cwd

# Load in EOL Chiroptera 20k image bundle
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Chiroptera_breakdown_download_000001.txt" #@param ["https://editors.eol.org/other_files/bundle_images/files/images_for_Chiroptera_breakdown_download_000001.txt"] {allow-input: true}
df = read_datafile(bundle)
print("EOL image bundle with {} images: \n{}".format(len(df), df.head()))

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]

# Display detection results on images?
display_results = "yes (use this option if testing tiny subsets; only works for \u003C50 images)" #@param ["yes (use this option if testing tiny subsets; only works for \u003C50 images)", "no (use this option if running batches)"]

# Take 5k subset of bundle for running inference
# Change filename for each batch
tags_file = "human_present_tags_d" #@param ["human_present_tags_a", "human_present_tags_b", "human_present_tags_c", "human_present_tags_d"] {allow-input: true}
tags_file = tags_file + ".txt"
imgs_dir = "data/imgs/"
outfpath = imgs_dir + tags_file
print("\nSaving tagging results to: \n{}".format(outfpath))

# Add 5k subset of image bundle urls as column in tags file
start, stop, cutoff = set_start_stop(run, df)
df = df.iloc[start:stop]
df.to_csv(outfpath, sep='\n', index=False, header=False)

In [ ]:
#@title Run inference for batches A-D
# Note: YOLO cannot parse images from URL, so images are temporarily downloaded
# Note: Takes 7-10 min per 5k imgs, aria2 downloads 16imgs at a time
%cd $imgs_dir
!aria2c -x 16 -s 1 -i $tags_file

# Check how many images downloaded
print("Number of files downloaded to Google Drive: ")
no_files = len([1 for x in list(os.scandir('.')) if x.is_file()])-1 # -1 because .txt file contains image filenames
if (no_files < cutoff) and ("tiny subset" not in run):
    print("\n\n\n \033[93m WARNING: Less than {} files were downloaded. This is likely due to broken EOL image bundle URLs.")

# Move tags file used for downloading images to data/img_info/
%cd $cwd
!mv data/imgs/*.txt data/img_info/

# Make a new list of successfully downloaded image files for running inference
inf_imgs = imgs_dir + tags_file
with open(inf_imgs, 'w', encoding='utf-8') as f:
    # Walk through data/imgs/ to list files
    for dir, dirs, files in os.walk(imgs_dir):
        files = [fn for fn in files]
        for fn in files:
            if 'txt' not in fn:
                out = "data/imgs/" + fn
                f.writelines(out + '\n')

# Inspect textfile of images for inference
df = pd.read_table(inf_imgs, header=None, sep='\r')
print("\nNumber of valid images ready for inference in {}: {}".format(inf_imgs, len(df)))

# Run darknet with flag to not show bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights -dont_show -save_labels < {outfpath}

print("\n\n~~~\n \033[92m Inference complete! Post-process inference results in next code blocks before running these steps for remaining batches A-D. \033[0m \n~~~")

In [ ]:
#@title Post-process detection results for batches A-D

# Combine individual prediction files for each image to all_predictions.txt
df = combine_predictions(imgs_dir, outfpath)

# Delete inference text files and images (only needed them for inference)
!rm -r data/imgs/*

# Add class names to numeric image tags
tags = add_class_names('data/results/all_predictions.txt')

# Add EOL media URL's from bundle to image tags df
final_tags = add_eolMediaURLs(tags, bundle)

# Save final tags to file
outpath = set_outpath(tags_file, cwd)
final_tags.to_csv(outpath, sep="\t", index=False)

print("\n\n~~~\n \033[93m Post-processing complete! Run above steps for remaining batches A-D before proceeding to next steps. \033[0m \n~~~")

## Combine tags from batches A-D
---
After running steps above for each image batch, combine tag files to one 20k tag dataset.

In [ ]:
#@title Define parameters for converting detected classes to desired image tags
%cd $cwd

# Write header row of output tagging file
# Enter any filename from 4 batches of tagging files
tags_file = "human_present_tags_a" #@param ["human_present_tags_a"] {allow-input: true}
tags_fpath = "data/results/" + tags_file + ".tsv"

# Combine exported model predictions and confidence values for all batches
df = combine_tag_files(tags_fpath)

# Filter for desired classes
# These classes will be converted to 'human present'
filter1 = ['Human', 'Person'] #@param
pattern1 = '|'.join(filter1)

# Remove all detections that aren't for filtered class(es)
df.loc[~df.tag.str.contains(pattern1), 'tag'] = 'None'
print("\nNo. tags not matching filtered classes: \n", len(df[~df.tag.str.contains(pattern1)]))
print("\nTags not matching filtered classes: \n", df[~df.tag.str.contains(pattern1)])

# Set all detections for 'Human' to 'Human present'
# (Human + body, eye, head, hand, foot, face, arm, leg ear, eye, face, nose, beard)print("\nNo. tags matching filtered class(es) {}: \n{}\n".format(filter, len(df[df.tag.str.contains(pattern1)])))
print("\nNo. tags matching filtered class(es) {}: \n{}\n".format(filter1, len(df[df.tag.str.contains(pattern1)])))
print("\nTags matching filtered class(es): \n", df[df.tag.str.contains(pattern1)])
df.loc[df.tag.str.contains(pattern1), 'tag'] = 'human present'

# Write results to tsv
outpath = 'data/results/' + tags_file.rsplit('_', 1)[0] + '_final.tsv'
df.to_csv(outpath, sep='\t', index=False)
print("\n\nFinal tagging file {}: \n{}".format(outpath, df.head()))

## Display tagging results on images
---

In [ ]:
#@title Adjust start index and display up to 50 images with their tags
# Suppress warning about too many figures opened
plt.rcParams.update({'figure.max_open_warning': 0})

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# Adjust start index using slider
start = 0 #@param {type:"slider", min:0, max:5000, step:50}
stop = min((start+50), len(df))

# Loop through images
for i, row in df.iloc[start:stop].iterrows():
    try:
        # Read in image
        url = df['eolMediaURL'][i]
        img = url_to_image(url)

        # Fetch image tag
        tag = df['tag'][i]

        # Display progress message after each image is loaded
        print('Successfully loaded {} of {} images'.format(i+1, (stop-start)))

        # Plot image with tag
        _, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(img)
        plt.axis('off')
        plt.title('{}) Tag: {}'.format(i+1, tag))

    except:
        print('Check if URL from {} is valid'.format(url))